In [64]:
import pandas as pd
import math
from collections import defaultdict

In [68]:
file=pd.read_csv('train1.txt')
file.describe()

,id,gender
count,119897.00000,119897.000000
mean,59949.00000,0.555569
std,34611.42695,0.496905
min,1.00000,0.000000
25%,29975.00000,0.000000
50%,59949.00000,1.000000
75%,89923.00000,1.000000
max,119897.00000,1.000000


In [69]:
file.drop('id',axis=1,inplace=True)  #inplace 在原数据集上修改
#file

In [70]:
name_male=file[file['gender']==1]
name_female=file[file['gender']==0]
#print(name_male.head())
print(name_female.head())
#计算男 女的  比例作为先验概率
total={'male':len(name_male),'female':len(name_female) }
print(total)

  name  gender
1   玉璎       0
3   越英       0
4   蕴萱       0
5   子颀       0
6   靖曦       0
{'male': 66611, 'female': 53286}


In [71]:
#  计算  名字里面每个字在 男  女  生里面出现的次数
list_name_fe=defaultdict(int)
list_names_fe=defaultdict(int)
for name in name_female['name']:
    list_names_fe.setdefault(name,0)
    list_names_fe[name]+=1
    for ch in name:
        list_name_fe.setdefault(ch,0)
        list_name_fe[ch]+=1
        
list_name_ma=defaultdict(int)
list_names_ma=defaultdict(int)
for name in name_male['name']:
    list_names_ma.setdefault(name,0)
    list_names_ma[name]+=1
    for ch in name:
        list_name_ma.setdefault(ch,0)
        list_name_ma[ch]+=1
        

print(list_name_ma['智'])

369


In [72]:
num=list_name_fe['t']
print(num)
num=list_name_ma['t']
print(num)

0
0


In [73]:
# 使用  拉普拉斯平滑  算出概率    (次数+1) /(所有的情况+total)
def lapulasi(char,num_list,total,alpha=1):
    count=num_list[char]
    charnum=len(num_list)
    smooth=(alpha+count)/(total+charnum)
    return smooth

In [74]:
print('智在男性的概率：',lapulasi('智',list_name_ma,total['male']) )
print('智在女性的概率：',lapulasi('智',list_name_fe,total['female']) )

智在男性的概率： 0.005171353496953094
智在女性的概率： 0.0012590714348168226


In [75]:
#转成对数的加法
def getLog(char,num_list,total,alpha=1):
    smooth=lapulasi(char,num_list,total,alpha)
    return math.log(smooth)

print(getLog('智',list_name_ma,total['male']))
print(getLog('智',list_name_fe,total['female']))

-5.264620826474866
-6.677380786199262


In [76]:
def combine_log(char,num_list_fe,nums_list_fe,num_list_ma,nums_list_ma,total):
    log_m=0
    log_f=0
    log_m+=getLog(char,nums_list_ma,total['male'])
    log_f+=getLog(char,nums_list_fe,total['female'])
    for ch in char:
        log_m+=getLog(ch,num_list_ma,total['male'])
        log_f+=getLog(ch,num_list_fe,total['female'])
    return {'male':log_m,'female':log_f}

In [77]:
choose=combine_log('雨彦',list_name_fe,list_names_fe,list_name_ma,list_names_ma,total)

In [78]:
def choose_sex(choose):
    if choose['male']>=choose['female']:
        return 1
    else :
        return 0
choose_sex(choose)

0

In [57]:
choose=combine_log('狗屎',list_name_fe,list_names_fe,list_name_ma,list_names_ma,total)
choose_sex(choose)

0

In [79]:
#测试读取一个测试文件
from numpy import *
def test(filename,total,list_name_fe,list_names_fe,list_name_ma,list_names_ma):
    train=pd.read_csv(filename)
    testTotal=shape(train)[0]
    results={}   #  存 混淆矩阵
    for index,data in train.iterrows():  #   iterrows  
        dataResult=choose_sex(combine_log(data[1],list_name_fe,list_names_fe,list_name_ma,list_names_ma,total) )
        realClass=data[2]
        results.setdefault(realClass,{})
        results[realClass].setdefault(dataResult,0)
        results[realClass][dataResult]+=1
    return (results,testTotal)

In [80]:
metrics,total_test=test('test1.txt',total,list_name_fe,list_names_fe,list_name_ma,list_names_ma)
print(metrics)
print(total)

{0: {0: 36, 1: 8}, 1: {0: 14, 1: 44}}
{'male': 66611, 'female': 53286}


In [60]:
train=pd.read_csv('test1.txt')
for index ,rows in train.head().iterrows():
    print('--------------')
    print(index)
    print('---------------------')
    print(rows)

--------------
0
---------------------
119898    119899
芸荭            妃酣
1              0
Name: 0, dtype: object
--------------
1
---------------------
119898    119900
芸荭            伟弟
1              0
Name: 1, dtype: object
--------------
2
---------------------
119898    119901
芸荭            沫阳
1              1
Name: 2, dtype: object
--------------
3
---------------------
119898    119902
芸荭            柏山
1              1
Name: 3, dtype: object
--------------
4
---------------------
119898    119903
芸荭            丁维
1              1
Name: 4, dtype: object


In [81]:
'''
                预测
              yes   no    合计
      yes     TP    FN     P
实际  no      FP    TN     N

合计          P     N      P+N

准确率   (TP+TN)/(P+N)
精确率   TP/(TP+FN)

        1      0
    1   9      4
    0   6      21


'''

#计算模型准确率
def showAccuracy(metrics):
    preRight=0  # TP + TN
    total=0
    for category,preDict in metrics.items():
        preRight+=metrics[category][category]
        for ca,result in preDict.items():
            total+=result
    return preRight/total

show_right=showAccuracy(metrics)
print(show_right)

0.7843137254901961


In [82]:
#计算模型精确率  对1的精确率   9/(9+6)    对0的精确率  21/(21+6)
def showPrecision( metrics):
    n=len(metrics)
    precision={}
    for i in range(n):
        ySum=sum(metrics[row][i]for row in range(n))
        p=metrics[i][i]/ySum
        precision[i]=p
    return precision

precision_test=showPrecision(metrics)
print(precision_test)

{0: 0.72, 1: 0.8461538461538461}


In [83]:
#计算召回率   对1的召回率  9/(9+4)   对0:  21/(21+6)
def showRecall(metrics):
    n=len(metrics)
    recall={}
    for i in range(n):
        xSum=sum(value for category,value in metrics[i].items())
        recall[i]=metrics[i][i]/xSum
    return recall

recall_test=showRecall(metrics)
print(recall_test)

{0: 0.8181818181818182, 1: 0.7586206896551724}


In [84]:
import csv
def testAndGenSubmitCsv(filename,total,list_name_fe,list_names_fe,list_name_ma,list_names_ma,genFileName):
    test=pd.read_csv(filename)
    #以写入的方式打开文件
    fileobj=open(genFileName,'w',newline ='')
    
    #初始化 
    writer=csv.writer(fileobj,dialect='excel')
    
    result={}
    for index,data in test.iterrows():  #   iterrows  
        dataResult=choose_sex(combine_log(data[1],list_name_fe,list_names_fe,list_name_ma,list_names_ma,total) )
        result[data[0]]=dataResult
    
    
    writer.writerow(['id','gender'])
    writer.writerows(result.items())
        
    fileobj.close()

In [45]:
testAndGenSubmitCsv('test.txt',total,list_name_fe,list_names_fe,list_name_ma,list_names_ma,'submit.csv')